# 400.1 | Prompt Engineering For Developers

Notes and exercises based on the [ChatGPT Prompt Engineering For Developers](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) course from DeepLearning.AI.

---

### 0. Validate Setup

Before you begin working on exercises in this section, make sure you are setup to work with the right LLM Providers by running the relevant code cells from the [LLM Setup](./400-00-llm-setup.ipynb) notebook.

---

### 1. Introduction 

This course was created when OpenAI was using version 0.28. In this notebook I hope to use the course for insights, but use it in conjunction with the OpenAI documentation to update the learnings for the latest OpenAI versions and practices. Quick study notes

1. LLM = Large Language Model. Understands Natural Language. Takes NLP inputs (prompts), returns NLP outputs (responses).
1. 2 types of LLM = Base LLM (predicts next word), Instruction Tuned LLM (follows instructions)
1. Base LLM = Default, also referred to as Foundation Models that are trained on large corpora of text data.
1. Instruction Tuned LLM = Fine-tuned version of base LLM using RLHF (Reinforcement Learning from Human Feedback) to follow instructions.
1. Generative AI Focus = Increasingly on Instruction Tuned LLMs, and using NLP to have AI execute tasks based on instruction prompts.

LLMs are smart in the sense of having a massive knowledge base and fast pattern-matching capabilities. But they lack the human cognitive ability to understand context, reason, and infer meaning. Instead, they are effectively reflecting the data they were trained on - which means instruction-tuned LLMs are only as effective as the clarity and specificity of instructions they are given.


The course has its own sandbox environment for practice, but I want to try my own experiments here. 
- Refer to the [Setup notebook](400-00-llm-setup.ipynb) for code snippets for chat completion
- Refer to the [OpenAI API documentation](https://platform.openai.com/docs/quickstart?context=python) for quickstarts
- Refer to the [OpenAI Cookbook](https://cookbook.openai.com/) for richer examples

---

### 2. Principle 1: Write Clear and Concise Instructions

#### 2.0 Helper Function

In [4]:
# Create an OpenAI client instance
from openai import OpenAI
client = OpenAI()

# Create a helper function to reuse across exercises
# So we can focus on prompt engineering more clearly
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt }]
  completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0, #degree of randomness
  )
  return completion.choices[0].message.content

# Prompting Principles
# 1. Write clear and specific instructions
# 2. Give the model time to "think"
# =====================================

#### 2.1 Tactic: Delimiters For Contextual Boundaries

Give model clear guidance on what it should focus on for the task. Delimiters can also reduce prompt injection attacks by guiding it to focus on the right context.

In [14]:

# Make sure cells are run in order to have client setup.
# The prompt (user or system message) defines delimiter.

text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
noise = f"""
Let's talk about amazing food that you can get when \
travel the world and explore different cultures. \
I love sushi in Japan and pasta in Italy. \
I love paella in Spain and tacos in Mexico. \
"""

# Craft Prompt
prompt = f"""
Summarize the text delimited by triple asterisks \ 
into 3 bullet points each with under 10 words.
{text}***{noise}***
"""

# Print Completion 
print(get_completion(prompt))

- Explore diverse cultures through food
- Enjoy sushi in Japan, pasta in Italy
- Try paella in Spain, tacos in Mexico


#### 2.2 Tactic: Ask For Structured Output

In [10]:
# Craft Prompt
prompt = f"""
Generate a list of three imaginary movies \ 
with a short teaser description, genre and movie rating. 
Provide them in JSON format with the following keys: 
movie_id, title, description, genre, rating.
"""

# Print Completion 
print(get_completion(prompt))

```json
[
    {
        "movie_id": 1,
        "title": "The Last Frontier",
        "description": "In a post-apocalyptic world, a group of survivors must navigate through dangerous territories to find a safe haven known as The Last Frontier.",
        "genre": "Action/Adventure",
        "rating": "PG-13"
    },
    {
        "movie_id": 2,
        "title": "Echoes of the Past",
        "description": "A detective uncovers a series of murders that seem to be connected to a dark secret from his own past, leading him on a dangerous journey to solve the mystery.",
        "genre": "Mystery/Thriller",
        "rating": "R"
    },
    {
        "movie_id": 3,
        "title": "Dreamscape",
        "description": "A young woman discovers she has the ability to enter people's dreams, but soon realizes that her gift comes with a dark side that threatens to consume her.",
        "genre": "Fantasy/Horror",
        "rating": "PG-13"
    }
]
```


#### 2.3 Tactic: Ask Model to Check Conditions Met


In [22]:
# Primary Content
good_text = f"""
To make a good almond cookie you need to buy a pack of almond flour \
and have 1 egg, 1 stick of butter and a cup of brown sugar on hand. \
Get a bowl and mix those ingredients with a pinch of salt and baking \
soda - but make sure you melt the butter first. Then wrap the dough \
and refrigerate for an hour. Preheat the oven to 350 degrees. Remove \
dough, flatten it out and cut out rounds. Bake for 10 minutes. \ 
Let them cool and enjoy your cookies!
"""
bad_text = f"""
It's a beautiful day outside. The sun is shining, the birds are singing. \
The flowers are blooming. On days like these, I want to bake cookies!
"""

# Craft Prompt
instruction = f"""
You will be provided with text delimited by four asterisks. \
If that texts contains a sequence of instructions rewrite in steps like: \

- First ... 
- Then ..
- Then ..
...
- Finally ..

Make sure each step has only 1 sentence. Break things down.
Use simple conversational language and be clear
If text has no instructions write \"Nothing to do!\"

****{good_text}{bad_text}****
"""

# Print Completion 
prompt = f"""
{instruction}****{good_text}****
"""
print(get_completion(prompt))
print("-------------")

# Print Completion 
prompt = f"""
{instruction}****{bad_text}****
"""
print(get_completion(prompt))
print("-------------")


First, buy almond flour, an egg, butter, and brown sugar. 
Then, mix those ingredients in a bowl with salt and baking soda after melting the butter. 
Then, wrap the dough and refrigerate for an hour. 
Next, preheat the oven to 350 degrees. 
Then, remove the dough, flatten it, and cut out rounds. 
Finally, bake for 10 minutes, let them cool, and enjoy your cookies!
-------------
Nothing to do!
-------------


#### 2.4 Tactic: Few Shot Prompting (by examples)

Give examples of what you want model to do and let is figure out what to do, then give it a task to do.


In [23]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

<grandparent>: Resilience is like a tree that bends in the \ 
strongest winds but never breaks; it is the \ 
ability to bounce back from adversity and \ 
continue to grow and thrive despite challenges.


---

### 3. Principle 2: Give the Model Time To Think


#### 3.1 Tactic: Specify steps required (chain of reasoning)

Give the model a series of steps it should take before it returns the response. Slow the model down so it puts more time into the computation instead of returning an immediate result. The added effort will improve the quality

In [31]:

# example 1: Give it time to think
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""

prompt1 = f"""
Perform the following actions: 
1 - Summarize text delimited by triple asterisks, in one SHORT sentence \
2 - Translate summary into Spanish.
3 - List each name in the Spanish summary.
4 - Output a json object that contains the following \
keys: spanish_summary, num_names.

Separate answers for each step with "----" lines.

Text:
***{text}***
"""
print("Completion 1:")
print(get_completion(prompt1))

# example 2: Specify Format
prompt2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by 
  <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the 
  following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
print("Completion 2:")
print(get_completion(prompt2))

Completion 1:


----
1 - Jack and Jill go on a quest to fetch water, but misfortune strikes as Jack trips and tumbles down a hill with Jill following suit, yet they return home with comforting embraces and continue exploring with delight.
2 - Jack y Jill van en una búsqueda para traer agua, pero la desgracia golpea cuando Jack tropieza y cae por una colina con Jill siguiendo el ejemplo, sin embargo, regresan a casa con abrazos reconfortantes y continúan explorando con alegría.
3 - Jack, Jill
4 - {
    "spanish_summary": "Jack y Jill van en una búsqueda para traer agua, pero la desgracia golpea cuando Jack tropieza y cae por una colina con Jill siguiendo el ejemplo, sin embargo, regresan a casa con abrazos reconfortantes y continúan explorando con alegría.",
    "num_names": 2
} 
----
Completion 2:
Summary: Jack and Jill, siblings, go on a quest to fetch water but encounter misfortune on the way back.

Translation: Jack et Jill, frère et sœur, partent en quête d'eau mais rencontrent un malheur sur le c

#### 3.2 Tactic: Instruct Model To Work it out (chain of thought)

Instead of telling model what to do (give it a chain of reasoning) ask it to tell us how it would solve the problem (chain of thought). This slows model down and makes it think before rushing to a conclusion (otherwise it may get the wrong response). Try it.

In [37]:
problem = f"""
Question:
```
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.
```
"""

student_solution = """
Student's Solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
"""

prompt1 = f"""
Determine if the student's solution is correct or not.
{problem}
{student_solution}
"""

prompt2= f"""
Your task is to determine if the student's solution \
is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```
{problem}
{student_solution}
Actual solution:
"""


# example 1: No chain of thought
print("Quick Response:")
print(get_completion(prompt1))

# example 2: No chain of thought
print("-------------------")
print("Measured Response:")
print(get_completion(prompt2))

Quick Response:


KeyboardInterrupt: 